In [1]:
import matplotlib, os
import numpy as np
import imageio.v3 as imageio
from IPython.display import clear_output
from shared.Render import collect_saved_frames, render_function_point
from shared.TestFunctions import TEST_FUNCTIONS
matplotlib.use('Agg')
np.random.seed(42)

#### The teaching-learning based optimization algorithm

In [2]:
def teaching_learning_optimization(func, rnge, pop_size=20, generations=30):
    lower, upper = rnge
    # Initialize population and evaluate fitness.
    positions = np.random.uniform(lower, upper, size=(pop_size, 2))
    fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
    best_index = np.argmin(fitness)
    best_position = positions[best_index].copy()
    best_value = fitness[best_index]
    improvement_id = 0
    yield "g0000_i00000", tuple(best_position)
    for generation in range(1, generations + 1):
        # Compute mean vector and identify current teacher.
        mean_vector = positions.mean(axis=0)
        teacher_index = np.argmin(fitness)
        teacher = positions[teacher_index].copy()
        TF = np.random.randint(1, 3)
        # Teacher phase adjusts each learner according to the teacher and population mean.
        for i in range(pop_size):
            r = np.random.rand(2)
            candidate = positions[i] + r * (teacher - TF * mean_vector)
            candidate = np.clip(candidate, lower, upper)
            candidate_value = func(tuple(candidate))
            if candidate_value < fitness[i]:
                positions[i] = candidate
                fitness[i] = candidate_value
                if candidate_value < best_value:
                    best_value = candidate_value
                    best_position = candidate.copy()
                    improvement_id += 1
                    yield f"g{generation:04d}_i{improvement_id:05d}", tuple(best_position)
        # Refresh mean vector and teacher after the teacher phase updates.
        mean_vector = positions.mean(axis=0)
        teacher_index = np.argmin(fitness)
        teacher = positions[teacher_index].copy()
        # Learner phase lets pairs of learners interact and move toward better peers.
        for i in range(pop_size):
            j = np.random.randint(pop_size - 1)
            if j >= i:
                j += 1
            Xi = positions[i]
            Xj = positions[j]
            r = np.random.rand(2)
            if fitness[i] < fitness[j]:
                candidate = Xi + r * (Xi - Xj)
            else:
                candidate = Xi + r * (Xj - Xi)
            candidate = np.clip(candidate, lower, upper)
            candidate_value = func(tuple(candidate))
            if candidate_value < fitness[i]:
                positions[i] = candidate
                fitness[i] = candidate_value
                if candidate_value < best_value:
                    best_value = candidate_value
                    best_position = candidate.copy()
                    improvement_id += 1
                    yield f"g{generation:04d}_i{improvement_id:05d}", tuple(best_position)
        # Maintain record of the best solution found so far.
        current_best_index = np.argmin(fitness)
        if fitness[current_best_index] < best_value:
            best_value = fitness[current_best_index]
            best_position = positions[current_best_index].copy()
            improvement_id += 1
            yield f"g{generation:04d}_i{improvement_id:05d}", tuple(best_position)

#### Run the teaching-learning based optimization algorithm and collect frames for multiple functions

In [ ]:
gif_folder_path = "../gifs/tlbo"
os.makedirs(gif_folder_path, exist_ok=True)

In [4]:
for func, params in TEST_FUNCTIONS.items():
    print(f"Test function: {func.__name__}")
    rnge = (params[0], params[1])
    resolution = params[2]
    for idx, point in teaching_learning_optimization(func, rnge):
        frame = render_function_point(func, rnge=rnge, resolution=resolution, point=point, iteration=idx)
        imageio.imwrite(os.path.join(gif_folder_path, f"{func.__name__}_{idx}.png"), frame)
    collect_saved_frames(gif_folder_path, f"{str.capitalize(func.__name__)}.gif", fps=5)
clear_output()

#### Visualize the teaching-learning based optimization algorithm on multiple functions

![Sphere](../gifs/tlbo/Sphere.gif)
![Ackley](../gifs/tlbo/Ackley.gif)
![Rastrigin](../gifs/tlbo/Rastrigin.gif)
![Rosenbrock](../gifs/tlbo/Rosenbrock.gif)
![Griewank](../gifs/tlbo/Griewank.gif)
![Schwefel](../gifs/tlbo/Schwefel.gif)
![Levy](../gifs/tlbo/Levy.gif)
![Michalewicz](../gifs/tlbo/Michalewicz.gif)
![Zakharov](../gifs/tlbo/Zakharov.gif)